In [1]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import riccati_jn, riccati_yn
from airy_props import *
import os


dirs = [n for n in os.listdir() if (os.path.isdir(n) and ((n[-2:]=='_p') or (n[-2:]=='_s')))]
# dirs = ['H_1_s', 'H_0_plus_s', 'H_0_minus_s', 'H_2_s', 'H_3_p']
for dir in dirs:
    r, hls, ddrls = load_matrices(dir)
    print(dir)
    emin = hls[-1,0,0]
    n = len(hls[0])
    shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
    vd = (hls-shift)/4.637

    hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
    emax = np.abs(np.array(hfunc(r[0])[-1,-1]))

    w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
    wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])

    es = np.array([1])
    smats = {}
    for i in range(n):
        emax = np.abs(np.array(hfunc(r[0])[i,i]))
        emin = np.abs(np.array(hfunc(r[-1])[i,i]))
        es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
    es = np.unique(np.round(np.sort(es)))
    sigmas = np.zeros((len(es), n, n))
    ecm = es*4.637+np.real(hls[-1,0,0])
    for ei in range(len(es)):
        e = es[ei]
        lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
        ks = np.sqrt(np.abs(np.diag(e*np.eye(n)-hfunc(r[-1]))))
        yf = np.zeros((lmax, n, n), dtype=complex)
        nt = np.sum(np.diag(vd[-1]) < e)
        for l in range(lmax):
           yf[l] = airy_imb_prop(r, w(r[:-1]+np.diff(r)/2, l, e), wp(r[:-1]+np.diff(r)/2, l), n)
        ytot = np.array([(yf[l]+np.matrix(yf[l]).H)/2 for l in range(lmax)])
        jmat = np.zeros((lmax, n, n))
        nmat = np.zeros((lmax, n, n))
        jpmat = np.zeros((lmax, n, n))
        npmat = np.zeros((lmax, n, n))
        for j in range(n):
            if j <= nt:
                prej, prejp = riccati_jn(lmax-1, np.real(ks[j])*r[-1])
                pren, prenp = riccati_yn(lmax-1, np.real(ks[j])*r[-1])
                jmat[:,j,j] = prej/np.sqrt(np.real(ks[j]))
                nmat[:,j,j] = pren/np.sqrt(np.real(ks[j]))
                jpmat[:,j,j] = np.sqrt(np.real(ks[j])) * prejp
                npmat[:,j,j] = np.sqrt(np.real(ks[j])) * prenp
            else:
                jmat[:,j,j], prejp = riccati_in(lmax-1, np.real(ks[j])*r[-1])
                nmat[:,j,j], prenp = riccati_kn(lmax-1, np.real(ks[j])*r[-1])
                jpmat[:,j,j] = ks[j] * prejp
                npmat[:,j,j] = ks[j] * prenp
        rmat = np.array([-np.matrix(npmat[l] - ytot[l] @ nmat[l]).I @ np.matrix(jpmat[l] - ytot[l] @ jmat[l]) for l in range(lmax)])
        smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
        print(smat[0])
        smats[f'E = {ecm[ei]}'] = smat
    np.savez(f'{dir}/Smatrix.npz', smats)
    print(f'Directory {dir} is done!')

H_2_p
[[-0.79378241-0.60820185j]]
[[-0.13512567-0.99082847j]]
[[0.80433647+0.59417408j]]
[[-0.81861432-0.57434363j]]
[[0.47663178+0.87910304j]]
[[0.28965141-0.95713221j]]
[[-0.97130192+0.2378499j]]
[[0.54644851+0.8374927j]]
[[0.72473819-0.68902435j]]
[[-0.7250896-0.68865454j]]
[[-0.73326572+0.67994219j]]
[[0.55402771+0.83249823j]]
[[0.94101832-0.33835563j]]
[[-0.0316458-0.99949915j]]
[[-0.9408291-0.3388814j]]
[[-0.70316528+0.71102643j]]
[[0.3005856+0.95375484j]]
[[0.97396279+0.22670793j]]
[[0.72279485-0.69106266j]]
[[-0.1423579-0.98981525j]]
[[-0.87072095-0.49177741j]]
[[-0.93377719+0.35785494j]]
[[-0.34863805+0.93725744j]]
[[0.44681882+0.89462447j]]
[[0.95052184+0.31065775j]]
[[0.90058885-0.43467198j]]
[[0.37131928-0.92850525j]]
[[-0.33029366-0.94387822j]]
[[-0.85793211-0.51376307j]]
[[-0.99104571+0.13352305j]]
[[-0.70545086+0.70875884j]]
[[-0.14691803+0.98914867j]]
[[0.45381353+0.89109667j]]
[[ 8.78568712e-01+0.47761453j -2.76758975e-04-0.00114181j]
 [-2.76758975e-04-0.00114181j  5.6